In [1]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file creates a figure showing the results
# of using GALFIT to model a foreground galaxy 
# covering part of the Sunburst Arc

In [2]:
import os
import glob

import numpy as np

import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.offsetbox import AnchoredText

In [3]:
def plot():

    '''
    Make a figure showing the results of using GALFIT to remove a foreground galaxy
    covering part of the Sunburst Arc
    '''

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    figs = f'{home}/figs'

    # Get the GALFIT products of the HST F275W and F814W observations
    f275w_files = glob.glob(f'{data}/hst/galfit_cutouts/f275w_result_*_v5_wcs.fits')
    f814w_files = glob.glob(f'{data}/hst/galfit_cutouts/f814w_result_*_v5_wcs.fits')

    # Define the center and size of a square in celestial coordinates that will be used 
    # to define a cutout area of the observations to plot
    center = SkyCoord(frame='fk5', ra='15h50m04.37s', dec='-78d10m59.75s')
    size = 3.2 * u.arcsecond

    # Define source masks for the F275W and F814W observations
    f275w_source_mask = (fits.getdata(f275w_files[0]) >= np.abs(np.median(fits.getdata(f275w_files[0]))))
    f814w_source_mask = (fits.getdata(f814w_files[0]) >= 8 * np.abs(np.median(fits.getdata(f814w_files[0]))))

    # Assemble the source masks into an array
    masks = np.array([f275w_source_mask, f814w_source_mask], dtype=bool)

    # Instantiate the figure and axes
    fig, ax = plt.subplots(3,2, figsize=(5,7.5), constrained_layout=True)

    # Adjust the spacing between the subplots to minimize the space between them
    plt.subplots_adjust(hspace=0, wspace=0)

    # Labels to annotate to each row of subplots
    row_labels = np.array(['Original', 'Model', 'Residual'], dtype=str)

    # Labels to annotate to each column of subplots
    column_labels = np.array(['F275W', 'F814W'], dtype=str)

    # For each set of files from each filter
    for i, files in enumerate([f275w_files, f814w_files]):

        # For each file in the set
        for j, file in enumerate(files):

            # Cutout an area from the file's data corresponding to the square area defined previously
            cutout = Cutout2D(fits.getdata(file), center, size, wcs=WCS(fits.getheader(file)))

            # Set the minimum and maximum values of the colormap
            vmin = np.median(fits.getdata(files[0])[~masks[i]]) - 2 * np.std(fits.getdata(files[0])[~masks[i]])
            vmax = np.median(fits.getdata(files[0])[~masks[i]]) + 100 * np.std(fits.getdata(files[0])[~masks[i]])

            # Plot the cutout data
            ax[j,i].imshow(cutout.data, origin='lower', cmap='viridis', norm=colors.PowerNorm(gamma=1/3, vmin=vmin, vmax=vmax))

            # Get the pixel dimensions of the cutout data
            shape = np.shape(cutout.data)

            # Set the pixel coordinate limits of the subplot. The '1.5 / 0.03' represents
            # the half-width of the subplot in pixels, since each subplot is 3 arcseconds
            # on a side and each pixel is 0.03 arcseconds
            ax[j,i].set_xlim(shape[0] / 2 - 1.5 / 0.03, shape[0] / 2 + 1.5 / 0.03)
            ax[j,i].set_ylim(shape[1] / 2 - 1.5 / 0.03, shape[1] / 2 + 1.5 / 0.03)

            # Set the ticks and tick labels of each coordinate axis
            ax[j,i].set_xticks([shape[0] / 2 - 1 / 0.03,
                                shape[0] / 2,
                                shape[0] / 2 + 1 / 0.03],
                            ['$+$1', '0', '$-$1'])
            ax[j,i].set_yticks([shape[1] / 2 - 1 / 0.03,
                                shape[1] / 2,
                                shape[1] / 2 + 1 / 0.03],
                            ['$-$1', '0', '$+$1'])
        
            # Set the aspect ratio of the subplot so that it appears square
            ax[j,i].set_aspect('auto', adjustable='datalim')

            # Add a label indicating the type of GALFIT product
            data_type_label = AnchoredText(row_labels[j], loc='upper right', frameon=False, prop=dict(color='white', fontweight='bold'))
            ax[j,i].add_artist(data_type_label)

            # Add a label indicating the HST filter
            filter_label = AnchoredText(column_labels[i], loc='upper left', frameon=False, prop=dict(color='white', fontweight='bold'))
            ax[j,i].add_artist(filter_label)

    # Add labels to each coordinate axis of the figure
    fig.supxlabel(r'$\Delta$ Right ascension (")', ha='center')
    ax[1,0].set_ylabel(r'$\Delta$ Declination (")', fontsize='large')

    # Set which sides of each subplot to add labels to ticks, and which direction the ticks should face
    ax[0,0].tick_params(labelbottom=False, labelleft=True, top=True, right=True, direction='in')
    ax[1,0].tick_params(labelbottom=False, labelleft=True, top=True, right=True, direction='in')
    ax[2,0].tick_params(labelbottom=True, labelleft=True, top=True, right=True, direction='in')
    ax[0,1].tick_params(labelbottom=False, labelleft=False, top=True, right=True, direction='in')
    ax[1,1].tick_params(labelbottom=False, labelleft=False, top=True, right=True, direction='in')
    ax[2,1].tick_params(labelbottom=True, labelleft=False, top=True, right=True, direction='in')

    # Save the figure
    fig.savefig(f'{figs}/foreground_galaxy_galfit_modeling_results.pdf', bbox_inches='tight')

In [ ]:
plot()